#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import isnan, when, count, isnull, expr, current_date, col, date_format, year, month

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true") \
                                  .option("header", "true") \
                                  .csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 1485774 bytes.
Wrote 3810301 bytes.
Wrote 57561 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

# popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale').orderBy('sum(totale_generale)', ascending=False).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[4]: [('codice_nuts_1', 'string'),
 ('descrizione_nuts_1', 'string'),
 ('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('sigla_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_genere_maschile', 'int'),
 ('totale_genere_femminile', 'int'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_1', 'descrizione_nuts_1', 'codice_nuts_2', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+------------------+-------------+---------------------+-------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|
+-------------+------------------+-------------+---------------------+-------------+
 ITI| Centro| ITI4| Lazio| LAZ|
 ITI| Centro| ITI3| Marche| MAR|
 ITI| Centro| ITI2| Umbria| UMB|
 ITI| Centro| ITI1| Toscana| TOS|
 ITG| Isole| ITG1| Sicilia| SIC|
 ITG| Isole| ITG2| Sardegna| SAR|
 ITH| Nord-Est| ITH5| Emilia-Romagna| EMR|
 ITH| Nord-Est| ITH1| Bolzano| BOL|
 ITH| Nord-Est| ITH4| Friuli Venezia Gi...| FVG|
 ITH| Nord-Est| ITH2| Trento| TRE|
 ITH| Nord-Est| ITH3| Veneto| VEN|
 ITC| Nord-Ovest| ITC1| Piemonte| PIE|
 ITC| Nord-Ovest| ITC2| Valle d'Aosta| VDA|
 ITC| Nord-Ovest| ITC3| Liguria| LIG|
 ITC| Nord-Ovest| ITC4| Lombardia| LOM|
 ITF| Sud| ITF1| Abruzzo| ABR|
 ITF| Sud| ITF2| Molise| MOL|
 ITF| Sud| ITF3| Campania| CAM|
 ITF| Sud| ITF5| Basilicata| BAS|
 ITF| Sud| ITF4| Puglia| PUG|
 ITF| Sud| ITF6| Calabria| CAL|
+-------------+------------------+-------------+---------------------+-------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe(['totale_genere_maschile', 'totale_genere_femminile', 'totale_generale']) \
                    .show()

+-------+----------------------+-----------------------+-----------------+
summary|totale_genere_maschile|totale_genere_femminile| totale_generale|
+-------+----------------------+-----------------------+-----------------+
 count| 210| 210| 210|
 mean| 138333.79047619048| 145673.29523809525|284007.0857142857|
 stddev| 151728.9786894938| 152578.20019731886|303588.4857882117|
 min| 457| 1307| 1764|
 max| 795710| 796399| 1592109|
+-------+----------------------+-----------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| null| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| null| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| null| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| null| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| null| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| null| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| null| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| null| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| null| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| null| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[10]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('isolamento_domiciliare', 'int'),
 ('totale_positivi', 'int'),
 ('variazione_totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('dimessi_guariti', 'int'),
 ('deceduti', 'int'),
 ('totale_casi', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_1', 'string'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1') \
                    .show(50)

Il numero di regioni è: 21

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 null| null| Lazio|
 null| null| Sicilia|
 null| null| Emilia-Romagna|
 null| null| Toscana|
 null| null| Puglia|
 null| null| Abruzzo|
 null| null| Sardegna|
 null| null| Umbria|
 null| null| Piemonte|
 null| null| P.A. Bolzano|
 null| null| Molise|
 null| null| Calabria|
 null| null| Friuli Venezia Gi...|
 null| null| Valle d'Aosta|
 null| null| P.A. Trento|
 null| null| Marche|
 null| null| Basilicata|
 null| null| Veneto|
 null| null| Campania|
 null| null| Lombardia|
 null| null| Liguria|
 1| null| P.A. Bolzano|
 172410| ITI| Umbria|
 500340| ITH| P.A. Bolzano|
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF1| Abruzzo|
 ITF| ITF3| Campania|
 ITF| ITF4| Puglia|
 ITG| ITG1| Sicilia|
 ITG| ITG2| Sardegna|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH2| P.A. Trento|
 ITH| ITH1| P.A. Bolzano|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1', 'codice_nuts_2')

data = popolazione_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1').show(num_regioni)

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'dimessi_guariti']) \
                  .show()

+-------+----------------------+------------------+--------------------+-----------------+
summary|ricoverati_con_sintomi| terapia_intensiva|totale_ospedalizzati| dimessi_guariti|
+-------+----------------------+------------------+--------------------+-----------------+
 count| 8988| 8988| 8988| 8988|
 mean| 710.4000890075656| 81.29372496662216| 791.6938139741878| 40867.8722741433|
 stddev| 1292.5532118318006|143.00589322648972| 1430.8251679041118|84973.39642601715|
 min| 0| 0| 0| 0|
 max| 12077| 1381| 13328| 710637|
+-------+----------------------+------------------+--------------------+-----------------+

In [0]:
dati_covid_regioni.describe(['isolamento_domiciliare', 'totale_positivi', 'variazione_totale_positivi', 'nuovi_positivi', 'deceduti', 'totale_casi', 'tamponi']) \
                  .show()

+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+
summary|isolamento_domiciliare| totale_positivi|variazione_totale_positivi| nuovi_positivi| deceduti| totale_casi| tamponi|
+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+
 count| 8988| 8988| 8988| 8988| 8988| 8988| 8988|
 mean| 11597.211059190031| 12388.90487316422| 50.35502892745883|441.12505562972854|2439.7059412550066| 55696.48509123275| 830756.6902536716|
 stddev| 21211.774938890743|22216.134347624353| 828.2009975915025| 855.761383377797| 4656.140807335724|104567.67922190507|1309806.7926228403|
 min| 0| 0| -21926| -229| 0| 0| 0|
 max| 155066| 164406| 10263| 11489| 32688| 796289| 9277607|
+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 Piemonte|2020-02-27 18:00:00| 2| 0| 2| 0| 2| -1| -1| 0| 0| 2| 156| ITC| ITC1|
 Liguria|2020-03-01 17:00:00| 12| 1| 13| 8| 21| -17| -17| 4| 0| 25| 121| ITC| ITC3|
 Liguria|2020-03-02 18:00:00| 12| 1| 13| 5| 18| -3| -3| 4| 0| 22| 121| ITC| ITC3|
 Sicilia|2020-03-02 18:00:00| 2| 0| 2| 3| 5| -2| -2| 2| 0| 7| 307| ITG| ITG1|
 Piemonte|2020-03-09 18:00:00| 222| 50| 272| 65| 337| -18| -10| 0| 13| 350| 1681| ITC| ITC1|
 Calabria|2020-04-17 17:00:00| 154| 7| 161| 658| 819| -28| -18| 99| 73| 991| 21657| ITF| ITF6|
 Sardegna|2020-05-04 17:00:00| 91| 9| 100| 553| 653| -36| -2| 545| 119| 1317| 28052| ITG| ITG2|
 Basilicata|2020-05-07 17:00:00| 48| 2| 50| 105| 155| -17| -16| 202| 26| 383| 16272| ITF| ITF5|
 Basilicata|2020-05-08 17:00:00| 48| 2| 50| 102| 152| -3| -1| 204| 26| 382| 16777| ITF| ITF5|
 Marche|2020-05-19 17:00:00| 144| 17| 161| 1967| 2128| -187| -3| 3561| 986| 6675| 89985| ITI| ITI3|
 Sardegna|2020-05-25 17:00:00| 49| 3| 52| 179| 231| -14| -2| 994| 129| 1354| 51073| ITG| ITG2|
 Sardegna|2020-06-09 17:00:00| 9| 1| 10| 44| 54| -2| -1| 1176| 131| 1361| 64272| ITG| ITG2|
 Campania|2020-06-12 17:00:00| 61| 2| 63| 283| 346| -264| -229| 3832| 430| 4608| 230551| ITF| ITF3|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT')

dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 1| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 1| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 10| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 10| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 7| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 11| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 9| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 15| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 23| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 22| 27| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 47| 28| 74| 1| 0| 0| 0| 0| 0| 0| 0| 0| 75| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 43| 16| 59| 0| 0| 0| 0| 0| 0| 0| 0| 0| 59| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 27| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 61| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 80| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF| ITF3| Campania|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[19]: [('data_somministrazione', 'string'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_maschile', 'int'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(50)

Il numero di regioni è: 21

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2|Valle d'Aosta / V...|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1|Provincia Autonom...|
 ITH| ITH4|Friuli-Venezia Gi...|
 ITH| ITH3| Veneto|
 ITH| ITH2|Provincia Autonom...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(num_regioni)

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4|Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['sesso_maschile', 'sesso_femminile', 'prima_dose', 'seconda_dose']) \
                             .show()

+-------+-----------------+-----------------+------------------+------------------+
summary| sesso_maschile| sesso_femminile| prima_dose| seconda_dose|
+-------+-----------------+-----------------+------------------+------------------+
 count| 40544| 40544| 40544| 40544|
 mean|189.9932912391476|256.2231896211523|315.76361483820045|130.45286602209944|
 stddev|535.6269316003377| 689.860409673434| 942.9014827323075| 513.7125411582991|
 min| 0| 0| 0| 0|
 max| 13122| 18414| 26492| 15065|
+-------+-----------------+-----------------+------------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+-----------------+----------------------+------------------------------+--------------------------+-----------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+-----------------+----------------------+------------------------------+--------------------------+-----------------+
 count| 40544| 40544| 40544| 40544| 40544| 40544| 40544| 40544| 40544| 40544|
 mean| 78.73322809786899| 21.54696132596685| 15.833440213101815|17.884495856353592|62.32022000789266|143.5668902920284| 7.794124901341752| 28.38723362273086| 61.89495363062352|8.254932912391476|
 stddev| 231.96681417989768| 67.71368762473674| 62.413416280276955| 224.1771323712477|636.4811783141071|902.0458184179746| 43.62044503140658| 155.5180778593222| 273.83596386545383|37.54587588051037|
 min| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 max| 4661| 2303| 2284| 13616| 26100| 31014| 1214| 3583| 5780| 2008|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+-----------------+----------------------+------------------------------+--------------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+

## Analisi

### Dati covid Regioni

In [0]:
dati_covid_regioni.show()

+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 Abruzzo|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| ITF| ITF1|
 Basilicata|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| ITF| ITF5|
 Calabria|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| ITF| ITF6|
 Campania|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| ITF| ITF3|
 Emilia-Romagna|2020-02-24 18:00:00| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| ITH| ITH5|
 Friuli Venezia Gi...|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| ITH| ITH4|
 Lazio|2020-02-24 18:00:00| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| ITI| ITI4|
 Liguria|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| ITC| ITC3|
 Lombardia|2020-02-24 18:00:00| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| ITC| ITC4|
 Marche|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| ITI| ITI3|
 Molise|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| ITF| ITF2|
 Bolzano|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| ITH| ITH1|
 Trento|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| ITH| ITH2|
 Piemonte|2020-02-24 18:00:00| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| ITC| ITC1|
 Puglia|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| ITF| ITF4|
 Sardegna|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| ITG| ITG2|
 Sicilia|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| ITG| ITG1|
 Toscana|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| ITI| ITI1|
 Umbria|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| ITI| ITI2|
 Valle d'Aosta|2020-02-24 18:00:00| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| ITC| ITC2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

#### Attuali Positivi per Regione

In [0]:
attuali_positivi_regione = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data','denominazione_regione', 'totale_positivi') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumnRenamed('totale_positivi', 'attuali_positivi') \
                                             .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd'))

popolazione = popolazione_regioni.groupBy('denominazione_regione') \
                                 .sum('totale_generale')

cond = [popolazione.denominazione_regione == attuali_positivi_regione.regione]
attuali_positivi_regione = attuali_positivi_regione.join(popolazione, cond, how='left') \
                                                   .drop('denominazione_regione') \
                                                   .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                   .orderBy('regione')

attuali_positivi_regione = attuali_positivi_regione.withColumn('positivi_per_100k', (attuali_positivi_regione.attuali_positivi / attuali_positivi_regione.popolazione * 100000).cast('int')) \
                                                   .orderBy('positivi_per_100k', ascending=False)


attuali_positivi_regione.show(21)

+----------+--------------------+----------------+-----------+-----------------+
 data| regione|attuali_positivi|popolazione|positivi_per_100k|
+----------+--------------------+----------------+-----------+-----------------+
2021-04-26| Campania| 91719| 5712143| 1605|
2021-04-26| Puglia| 48728| 3953305| 1232|
2021-04-26| Sardegna| 17760| 1611621| 1101|
2021-04-26| Basilicata| 6064| 553254| 1096|
2021-04-26| Emilia-Romagna| 48523| 4464119| 1086|
2021-04-26| Lazio| 46623| 5755700| 810|
2021-04-26| Calabria| 14871| 1894110| 785|
2021-04-26| Abruzzo| 9057| 1293941| 699|
2021-04-26|Friuli Venezia Gi...| 8265| 1206216| 685|
2021-04-26| Valle d'Aosta| 814| 125034| 651|
2021-04-26| Toscana| 22671| 3692555| 613|
2021-04-26| Sicilia| 26091| 4875290| 535|
2021-04-26| Lombardia| 52964| 10027602| 528|
2021-04-26| Veneto| 23507| 4879133| 481|
2021-04-26| Marche| 6412| 1512672| 423|
2021-04-26| Liguria| 6068| 1524826| 397|
2021-04-26| Piemonte| 17089| 4311217| 396|
2021-04-26| Umbria| 2985| 870165| 343|
2021-04-26| Molise| 639| 300516| 212|
2021-04-26| Trento| 1160| 545425| 212|
2021-04-26| Bolzano| 802| 532644| 150|
+----------+--------------------+----------------+-----------+-----------------+

In [0]:
# TODO: CONTINUARE/AGGIUSTARE

dati_covid_regioni.select(date_format('data','yyyy-MMMM').alias('mese'), 'deceduti').groupby('mese').sum('deceduti').withColumnRenamed('sum(deceduti)', 'deceduti').orderBy('deceduti', ascending=False).show()

+--------------+--------+
 mese|deceduti|
+--------------+--------+
 2021-March| 3201000|
 2021-April| 2990863|
 2021-February| 2622049|
 2021-January| 2531790|
 2020-December| 2046231|
 2020-November| 1385098|
 2020-October| 1137835|
 2020-August| 1095146|
 2020-July| 1084927|
2020-September| 1069879|
 2020-June| 1028841|
 2020-May| 973585|
 2020-April| 644171|
 2020-March| 116566|
 2020-February| 96|
+--------------+--------+

In [0]:
dati_covid_regioni.select(date_format('data','yyyy-MMMM').alias('month')).groupby('month').count().show()

+-----------------+-----+
 month|count|
+-----------------+-----+
 2020-April-22| 21|
 2021-March-03| 21|
2020-September-11| 21|
2020-September-07| 21|
 2020-November-25| 21|
 2020-December-25| 21|
 2021-February-07| 21|
 2021-January-16| 21|
 2020-May-24| 21|
2020-September-17| 21|
 2021-March-22| 21|
 2020-April-30| 21|
 2021-March-15| 21|
 2021-March-14| 21|
 2021-April-08| 21|
 2021-April-12| 21|
 2021-April-16| 21|
 2020-March-31| 21|
 2020-December-08| 21|
 2020-March-01| 21|
+-----------------+-----+
only showing top 20 rows

### Dati Somministrazioni Vaccini

#### Percentuale popolazione vaccinata per regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 286804| 113241| 1293941| 22.16515281608667| 8.75163550733766|
 Basilicata| 113966| 51802| 553254| 20.59921844216219| 9.36314965639652|
 Bolzano| 130158| 49071| 532644| 24.43620879987384| 9.212719940523126|
 Calabria| 342582| 149402| 1894110| 18.08670035003247| 7.887715074626078|
 Campania| 1135920| 426662| 5712143| 19.886056774138883| 7.469385832952711|
 Emilia-Romagna| 1025147| 474847| 4464119| 22.964150373231536| 10.636970027008688|
Friuli Venezia Gi...| 266447| 121448| 1206216| 22.08949309244779| 10.068511775668702|
 Lazio| 1228450| 538975| 5755700| 21.343190228816653| 9.3641954931633|
 Liguria| 389123| 158705| 1524826| 25.519173990999626| 10.408072789944557|
 Lombardia| 2084135| 852818| 10027602| 20.783982052738033| 8.504705312396723|
 Marche| 375926| 129812| 1512672| 24.851785449852976| 8.58163567514967|
 Molise| 72552| 34293| 300516| 24.142474943097874| 11.411372439404225|
 Piemonte| 1001904| 419856| 4311217| 23.239470432594786| 9.738688634786882|
 Puglia| 891423| 289900| 3953305| 22.54880410188437| 7.333104832538851|
 Sardegna| 330768| 138710| 1611621| 20.523932115553222| 8.60686228337804|
 Sicilia| 905143| 405218| 4875290| 18.565931462538636| 8.311669664778917|
 Toscana| 830883| 331251| 3692555| 22.5015741133172| 8.97078039460482|
 Trento| 119947| 41570| 545425| 21.99147453820415| 7.621579502223037|
 Umbria| 201863| 79204| 870165| 23.198244011193278| 9.102181770124057|
 Valle d'Aosta| 30411| 10641| 125034| 24.32218436585249| 8.510485148039733|
 Veneto| 1038768| 471655| 4879133| 21.29001197548827| 9.666778913384816|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale popolazione vaccinata per fascia anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 23312| 7799| 2298846| 1.0140740180072958| 0.3392571751217785|
 20-29| 536636| 273815| 6084382| 8.819893294010797| 4.500292716663747|
 30-39| 783792| 365142| 6854632| 11.434486927963455| 5.326938047148264|
 40-49| 1140888| 465258| 8937229| 12.765567493011536| 5.205841765943337|
 50-59| 1560143| 632613| 9414195| 16.57224011187361| 6.7197779523368695|
 60-69| 1843108| 505214| 7364364| 25.027388651620154| 6.86025296957076|
 70-79| 3127348| 407994| 5968373| 52.398668782932965| 6.8359333439783345|
 80-89| 3100342| 2145258| 3628160| 85.45218512965249| 59.12798774034221|
 90+| 686751| 485988| 791543| 86.76104772577105| 61.397548838155345|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale popolazione vaccinata per regione e fascia anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)

+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 16-19| 612| 187| 46751| 1.3090629077452889| 0.3999914440332827|
 Abruzzo| 20-29| 11441| 4705| 130083| 8.79515386330266| 3.6169215039628546|
 Abruzzo| 30-39| 17884| 7765| 150713| 11.866262366219239| 5.1521766536397|
 Abruzzo| 40-49| 27656| 9242| 190060| 14.551194359675893| 4.862674944754288|
 Abruzzo| 50-59| 40538| 11570| 204157| 19.85628707318387| 5.667207100417816|
 Abruzzo| 60-69| 50153| 10102| 167705| 29.905488804746426| 6.02367252019916|
 Abruzzo| 70-79| 56130| 4773| 130572| 42.9877768587446| 3.655454461906075|
 Abruzzo| 80-89| 67416| 53035| 84488| 79.79358015339456| 62.772228008711295|
 Abruzzo| 90+| 14974| 11862| 19515| 76.73071995900588| 60.78401229823213|
 Basilicata| 16-19| 300| 28| 22032| 1.3616557734204793| 0.1270878721859114|
 Basilicata| 20-29| 3731| 1170| 61183| 6.098099145187389| 1.912295899187683|
 Basilicata| 30-39| 6694| 2764| 65204| 10.26624133488743| 4.239003742101712|
 Basilicata| 40-49| 11343| 3815| 78467| 14.455758471714224| 4.861916474441485|
 Basilicata| 50-59| 16621| 4723| 87822| 18.925781694791738| 5.377923527134431|
 Basilicata| 60-69| 18842| 3812| 72817| 25.875825700042572| 5.235041267835808|
 Basilicata| 70-79| 17924| 1061| 51805| 34.598976932728505| 2.048064858604382|
 Basilicata| 80-89| 30934| 27882| 36107| 85.67313817265351| 77.22048356274406|
 Basilicata| 90+| 7577| 6547| 7823| 96.85542630704333| 83.68912182027356|
 Bolzano| 16-19| 336| 123| 23269| 1.4439812626240922| 0.5286002836391765|
 Bolzano| 20-29| 5776| 2757| 61094| 9.454283563034013| 4.512718106524372|
 Bolzano| 30-39| 8078| 3455| 63127| 12.796426251841526| 5.473093921776736|
 Bolzano| 40-49| 12220| 5199| 75664| 16.150348910974834| 6.871167265806724|
 Bolzano| 50-59| 16162| 6602| 83539| 19.346652461724464| 7.902895653527095|
 Bolzano| 60-69| 29787| 3784| 57041| 52.220332743114604| 6.633824792692976|
 Bolzano| 70-79| 30632| 3964| 46613| 65.71557290884517| 8.50406538948362|
 Bolzano| 80-89| 22108| 18708| 27108| 81.55526043972259| 69.01283753873395|
 Bolzano| 90+| 5059| 4479| 6165| 82.06001622060016| 72.65206812652069|
 Calabria| 16-19| 691| 248| 76743| 0.9004078547880588| 0.3231565093884784|
 Calabria| 20-29| 12504| 5621| 213889| 5.846022937130941| 2.627998634805904|
 Calabria| 30-39| 21875| 11069| 236668| 9.242905673770851| 4.677015904135751|
 Calabria| 40-49| 32314| 13665| 266417| 12.12910587537582| 5.129177192146147|
 Calabria| 50-59| 47216| 19023| 286804| 16.462810839458307| 6.632752681273622|
 Calabria| 60-69| 70774| 21684| 241215| 29.340629728665295| 8.98949070331447|
 Calabria| 70-79| 67306| 13610| 175208| 38.41491256107027| 7.767910141089448|
 Calabria| 80-89| 74410| 53706| 107720| 69.0772372818418| 49.85703676197549|
 Calabria| 90+| 15492| 10776| 23058| 67.18709341660161| 46.734322144158206|
 Campania| 16-19| 3075| 912| 261694| 1.1750364930032786| 0.3484986281687773|
 Campania| 20-29| 50065| 22772| 693479| 7.21939669405995| 3.2837331772122877|
 Campania| 30-39| 76647| 33799| 715258| 10.715993389797806| 4.725427747749763|
 Campania| 40-49| 110905| 39313| 835597| 13.272546454810152| 4.70477993578244|
 Campania| 50-59| 163071| 54697| 868684| 18.77218873606513| 6.296535909490678|
 Campania| 60-69| 238238| 59766| 670867| 35.51195691545418| 8.9087702927704|
 Campania| 70-79| 266543| 39475| 484380| 55.02766423056278| 8.14959329452083|
 Campania| 80-89| 193556| 153391| 255273| 75.82313836559291| 60.089002753914436|
 Campania| 90+| 33820| 22537| 49044| 68.95848625723839| 45.95261397928391|
 Emilia-Romagna| 16-19| 1933| 669| 160045| 1.207785310381455| 0.41800743540879126|
 Emilia-Romagna| 20-29| 43380| 23694| 42066

# IDEE
* mesi/regioni più critiche dal punto di vista dei contagi/TI/casi/morti
* positivi (o casi) per ogni 100k abitanti per ogni regione
* affiancare alla percentuale di seconde dosi la percentuale di posti letto occupati in TI per regione (aggiungere tabella)
* valutare possibilità di aggiungere tabella posti letto e consegne vaccini